In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

In [ ]:
# Load the Drive helper and mount
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Path to your dataset
dataset_path = '/content/drive/MyDrive/Dataset_IC01/Gigi Tampak Depan'

In [ ]:
# Image dimensions
img_height, img_width = 224, 224
batch_size = 32

In [ ]:
# Prepare the dataset
datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

train_generator = datagen.flow_from_directory(
    dataset_path,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)

validation_generator = datagen.flow_from_directory(
    dataset_path,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)

Found 6191 images belonging to 5 classes.
Found 1545 images belonging to 5 classes.


In [ ]:
# Load the VGG16 model pre-trained on ImageNet, excluding the top layers
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(img_height, img_width, 3))


58889256/58889256 [==============================] - 0s 0us/step


In [ ]:
# Freeze the layers of VGG16
for layer in base_model.layers:
    layer.trainable = False

In [ ]:
# Add custom layers on top of VGG16
model = Sequential([
    base_model,
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(train_generator.num_classes, activation='softmax')
])

In [ ]:
# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
# Print model summary
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 7, 7, 512)         14714688  
                                                                 
 flatten (Flatten)           (None, 25088)             0         
                                                                 
 dense (Dense)               (None, 512)               12845568  
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                                 
 dense_1 (Dense)             (None, 5)                 2565      
                                                                 
Total params: 27562821 (105.14 MB)
Trainable params: 12848133 (49.01 MB)
Non-trainable params: 14714688 (56.13 MB)
_________________________________________________________________


In [ ]:
# Define callbacks
early_stopping = EarlyStopping(monitor='val_accuracy', patience=10, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.001)

class CustomCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        if logs.get('accuracy') >= 0.98 and logs.get('val_accuracy') >= 0.98:
            print("\nReached 98% accuracy and validation accuracy, stopping training.")
            self.model.stop_training = True

callbacks = [CustomCallback(), early_stopping, reduce_lr]

In [ ]:
# Train the model
model.fit(
    train_generator,
    epochs=30,  # Set a high number to allow early stopping
    validation_data=validation_generator,
    callbacks=callbacks
)

Epoch 1/50
194/194 [==============================] - 5051s 26s/step - loss: 1.2836 - accuracy: 0.5508 - val_loss: 0.8275 - val_accuracy: 0.6408 - lr: 0.0010
Epoch 2/50
194/194 [==============================] - 4968s 26s/step - loss: 0.8057 - accuracy: 0.6610 - val_loss: 0.7083 - val_accuracy: 0.7042 - lr: 0.0010
Epoch 3/50
194/194 [==============================] - 4885s 25s/step - loss: 0.7129 - accuracy: 0.7025 - val_loss: 0.6870 - val_accuracy: 0.7353 - lr: 0.0010
Epoch 4/50
194/194 [==============================] - 4864s 25s/step - loss: 0.6290 - accuracy: 0.7419 - val_loss: 0.6411 - val_accuracy: 0.7405 - lr: 0.0010
Epoch 5/50
179/194 [==========================>...] - ETA: 5:01 - loss: 0.5799 - accuracy: 0.7680

In [ ]:
# Evaluate the model
loss, accuracy = model.evaluate(validation_generator)
print(f'Validation accuracy: {accuracy:.2f}')

In [ ]:
# Save the model
model.save('vgg_image_classifier.h5')